In [64]:
import os
import numpy as np
import pandas as pd

os.chdir('..')

In [105]:
import Configs.config as config
cutoff_date = np.datetime64('2016-04-09')

In [142]:
sp = pd.read_csv(config.PATH + 'Data\\Yahoo\\^GSPC.csv')
sp[['Open', 'High', 'Low', 'Close', 'Adj Close']] = sp[['Open', 'High', 'Low', 'Close', 'Adj Close']].div(sp.iloc[0]['Open'])
sp['Date'] = pd.to_datetime(sp['Date'])

In [107]:
spinoffs = pd.read_html('https://www.stockspinoffs.com/recent-spinoffs')[0]

In [108]:
spinoffs.head()

,Parent Name,Parent Ticker,Spinoff Name,Spinoff Ticker,Spinoff Date,More Info
0,Novartis,NVS,Alcon,ALC,"April 9, 2019",info
1,AP Moller Maersk,AMKBY,The Drilling Company of 1972 A/S,Copenhagen:DRLCO,"April 4, 2019",info
2,DowDupont (Merger of Dow Chemical & DuPont) TB...,DWDP,Dow Chemical Company,DOW,"April 1, 2019",info
3,Ironwood Pharmaceuticals,IRWD,Cyclerion Therapeutics,CYCN,"April 1, 2019",info
4,Capital Product Partners LP,CPLP,Diamond S Shipping,DSSI,"March 29, 2019",None Yet


In [156]:
spinoffs['Date'] = pd.to_datetime(spinoffs['Spinoff Date'], errors='coerce')
#Dropping spinoffs that occured in multiple parts for simplicity
spinoffs.dropna(inplace=True)
#Dropping spinoffs without at least a year of follow up data
spinoffs.drop(spinoffs[spinoffs['Date'] > cutoff_date].index, inplace=True)
spinoffs.shape

KeyError: False

In [150]:
results = pd.DataFrame(columns=spinoffs['Spinoff Ticker'])
for index, row in spinoffs.iterrows():
    stock = row['Spinoff Ticker']
    start = row['Date']
    try:
        df = pd.read_csv(config.PATH + 'Data\\Kaggle\\stocks\\' + stock + '.us.txt')
    except FileNotFoundError:
        print("No file: {}".format(stock))
        continue
    df['Date'] = pd.to_datetime(df['Date'])
    end_date = start + np.timedelta64(2, 'Y')
    df = df.drop(df[df['Date'] < start].index, axis=0)
    df = df.drop(df[end_date < df['Date']].index, axis=0).reset_index()
    benchmark = sp.drop(sp[sp['Date'] < start].index).drop(sp[end_date < sp['Date']].index).reset_index()
    benchmark = benchmark['Adj Close'].div(benchmark.iloc[0]['Open'])
    returns = df['Close'] / df.iloc[0]['Open'] / benchmark
    results[stock] = returns
    

No file: MFS
No file: OTC:TLSSF
No file: CSRA (ACQUIRED)
No file: CCP (ACQUIRED)
No file: OTC:AXDDF
No file: BXLT (ACQUIRED)
No file: CPGX (ACQUIRED)
No file: LSE:LMI
No file: TLN
No file: CSAL
No file: JMG (ACQUIRED)
No file: REMY (ACQUIRED)
No file: LSE:INDV
No file: KLX (ACQUIRED)
No file: LBRDA & LBRDK
No file: LTRPA & LTRPB
No file: TPUB
No file: NAME (ACQUIRED)
No file: PGN (BANKRUPT)
No file: SSE
No file: NSAM
No file: TIME (ACQUIRED)
No file: HAWKB
No file: SWAY (ACQUIRED)
No file: HEL:VALMT
No file: ASX:ORA
No file: ASX:REC (ACQUIRED)
No file: HART
No file: Milan:WDF
No file: STRP (ACQUIRED)
No file: Munich:OSR
No file: Paris:FNAC
No file: CST (ACQUIRED)
No file: OTC:CWGL
No file: AXLL (ACQUIRED)
No file: LMCAD or LMCBD. Will be LMCA and LMCB


In [154]:
print(np.average(results.iloc[250].dropna()))

0.9677028169838868


In [157]:
print(spinoffs)

                                           Parent Name  \
84                          Armstrong World Industries   
85                                          Greatbatch   
86                                   Manitowoc Company   
87                                      W.R Grace & Co   
89                                America Movil S.A.B.   
90                                     GAMCO Investors   
91                       Computer Sciences Corporation   
93                                  Darden Restaurants   
94                   Archrock (FKA: Exterran Holdings)   
95               HP Inc (FKA: Hewlett-Packard Company)   
96                                    Blackstone Group   
97   MSG Networks (FKA: Madison Square Garden Company)   
98                       Capital Southwest Corporation   
99                Fidelity National Financial Ventures   
100                                    SPX Corporation   
101                                             Ventas   
102           